This notebook demonstrates how to generate persistence images for a graph dataset, and evaluate the classification performance when using these images as features for a Random Forest.

In [1]:
from os import linesep
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

from GCTDA.datasets import load_dataset
from GCTDA.images import generate_image_features

In [2]:
datasets_dir = "tests/Datasets/preprocessed"

#Choose the dataset. Can be any dataset in the datasets directory
dataset_name = "MUTAG" 

#Parameters for computing the peristence diagrams and persistence images
filtration = "degree"
extended = False # True for extended persistence, False for ordinary
dimensions = [0,1] # 0 for connected components, 1 for cycles
weight = "uniform" # weight for persistence images; can be either 'uniform' or 'linear'
pixels = [7,7] # resolution of the persistence images
spread = 0.2 # std of gaussians for persistence images

In [3]:
np.random.seed(1)

# Load the dataset
graphs,y = load_dataset(dataset_name,directory=datasets_dir)


In [115]:
le = preprocessing.LabelEncoder()
encoded = lb.fit_transform(y)
y_encoded = encoded.reshape((encoded.shape[0],)) if encoded.shape[1] == 1 else np.argmax(encoded, axis=1)
y_encoded

array([0, 0, 0, ..., 2, 2, 2])

In [5]:
## Evaluate the classification performance with Random Forests

# param_grid = [{ "n_estimators": [100], "max_depth":[3,5,10,30,50]}]


## Perform 10 times a nested 10-fold CV, with an inner 5-fold CV for hyperparameter tuning
# mean_accuracies=[]
# for CV_seed in range(10):
#   cv = StratifiedKFold(n_splits=10, shuffle=True,random_state=CV_seed)
#   clf = GridSearchCV(RandomForestClassifier(random_state=CV_seed+10),
#                      param_grid=param_grid,scoring = "accuracy",n_jobs=-1,cv=5)
#   scores = cross_val_score(clf,X=X,y=y,cv=cv)
#   mean_accuracies.append(np.mean(scores))
    
# print("{:.1f}\u00B1{:.1f}".format(np.mean(mean_accuracies)*100,np.std(mean_accuracies)*100))

In [118]:
from pyper.persistent_homology import calculate_persistence_diagrams

from GCTDA.extendedPersistence import compute_extended_persistence_diagrams
from GCTDA.filtrations import calculate_filtration, scale_filtration_quantile, add_random_noise

def generate_persistence_diagrams_fn(graphs,filtration="degree",extended=False,dimensions=[0,1],weighting_type="uniform",spread=1,pixels=[7,7]):
    """From a graph dataset, generate a list of persistence diagrams.

    Parameters
    ----------
    graphs:
       A list of igraph graphs.
    filtration:
        Method to compute the filtration.
    extended:
        If True, use extended persistence. Otherwise, ordinary persistence is used.
    dimensions:
        When extended is False, specifies the dimensions of the components to use.
        Can be [0] for connected components, [1] for cycles or [0,1] for both.
    weighting_type:
        weight applied to the persistence image. Can be 'linear' (as in the original
        paper) or 'uniform' to ignore the weighting.
    spread:
        Standard deviation of gaussian kernel
    pixels:
        Resolution of the persistence image
    """

    # Compute the filtration
    graphs_filtration=[]
    for graph in graphs:
        graph_filtration = calculate_filtration(graph,method=filtration,attribute_out="f")
        graphs_filtration.append(graph_filtration)

    # Scale the filtration values, so that they are between 0 and 1.
    # I use a quantile transformation so that the values are uniformly distributed between 0 and 1.
    # Otherwise, if a few nodes have a very high value, then all the other values would be almost identical
    # Ex: if one node has a degree of 1000 and most of the nodes have a degree smaller than 10,
    # then with a linear scaling most nodes would have a value <0.01, and only one pixel of the persistence image would be used.
    graphs = scale_filtration_quantile(graphs_filtration,"f")

    # Add random noise to the filtration values, to ensure that all vertices have a different filtration value
    # This is a little "hack" to make sure that all vertices have a corresponding persistence tuple, even when using gudhi
    graphs = add_random_noise(graphs,"f")
    
    # Compute persistence diagrams
    # There are 4 diagrams in case of extended persistence. For ordinary persistence, there are 1 or 2, depending on dimensions.
    persistence_diagrams = [[] , [] ,[] , []] if extended else [[] for dim in dimensions]
    for graph in graphs:
        if extended:
            diagrams = compute_extended_persistence_diagrams(graph,attribute="f")
            for i in range(4):
                if i%2==1: # reverse(birth/death) for diagrams where the death occurs before the birth
                    diagram = [(v,u) for (u,v) in diagrams[i]]
                else:
                    diagram = diagrams[i]
                persistence_diagrams[i].append(diagram)
        else:
            pd_0, pd_1 = calculate_persistence_diagrams(graph,vertex_attribute='f', edge_attribute='f')
            for i, dim in enumerate(dimensions):
                if dim==0:
                    persistence_diagrams[i].append(pd_0._pairs)
                else:
                    persistence_diagrams[i].append(pd_1._pairs)
    return persistence_diagrams

pd = generate_persistence_diagrams_fn(graphs, filtration = filtration, extended = extended, dimensions = dimensions,
                                      weighting_type = weight, spread = spread, pixels = pixels)

In [119]:
pd[0][0][0]

(0.32740165384429465, 0.32744324647113776)

In [120]:
for dim in dimensions:
    for index, diagram in enumerate(pd[dim]):
        with open(dataset_name + "_" + str(f"{index:07d}") + "_" + str(y[index]) + "_" + str(dim) + ".txt", "w") as f:
            for event in diagram:
                f.write(str(event[0]) + " " + str(event[1]))
                f.write(linesep)